In [83]:
import pandas as pd
df = pd.read_csv("Mixed_Dataset.csv")

In [84]:
df_reduced = df.sample(n=100000, random_state=42) 
df_reduced = df_reduced.reset_index(drop=True)

In [91]:
df_reduced

,time,Latitude,longitude,Sender,Isnegation,Severity,Causecode,Subcausecode,TTL,Reliability,Hopcount,receiver,DENMLABEL,Event
0,17.531328,331.072,1692.37,31,0,4,1,2,22000,7,5,36,L,E1
1,35.447265,530.805,1622.81,22,1,9,2,8,40000,10,5,39,L,E3
2,23.168083,671.610,1907.01,21,0,4,1,2,28000,7,5,32,L,E1
3,12.108796,242.349,1853.39,42,0,4,1,2,17000,7,2,23,L,E1
4,33.658161,530.805,1622.81,40,1,9,2,8,38000,10,5,29,L,E3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,10.474756,870.331,1874.35,35,0,4,6,2,15000,7,5,38,M,E1
99996,27.533109,264.230,1821.82,20,0,4,1,2,32000,7,5,12,L,E1
99997,17.514809,265.589,1809.39,18,0,4,1,2,22000,7,5,6,L,E1
99998,33.615543,530.805,1622.81,25,1,9,2,8,38000,10,5,55,L,E3


In [97]:
df_reduced = df_reduced.drop(['Latitude','longitude','Isnegation','Severity','Causecode','Subcausecode','TTL','Reliability'], axis=1)

In [99]:
df_reduced

,time,Sender,Hopcount,receiver,DENMLABEL,Event
0,17.531328,31,5,36,L,E1
1,35.447265,22,5,39,L,E3
2,23.168083,21,5,32,L,E1
3,12.108796,42,2,23,L,E1
4,33.658161,40,5,29,L,E3
...,...,...,...,...,...,...
99995,10.474756,35,5,38,M,E1
99996,27.533109,20,5,12,L,E1
99997,17.514809,18,5,6,L,E1
99998,33.615543,25,5,55,L,E3


In [105]:
df_reduced = df_reduced.sort_values(by='time').reset_index(drop=True)

In [117]:
# Filter rows with Hopcount 1 and DENMLABEL 'M' 
hop1_m_df = df_reduced[(df_reduced['Hopcount'] == 1) & (df_reduced['DENMLABEL'] == 'M')]

# Reset index if needed
hop1_m_df = hop1_m_df.reset_index(drop=True)

print(hop1_m_df)

        time  Sender  Hopcount  receiver DENMLABEL Event
0   8.000240      29         1         4         M    E1
1   9.000240      29         1        25         M    E1
2   9.000241      29         1        35         M    E1
3   9.000241      29         1        28         M    E1
4  10.000234      29         1        35         M    E1
5  10.000528      39         1        31         M    E1
6  10.000528      39         1        31         M    E1
7  15.000822      59         1        56         M    E1
8  15.000823      59         1         3         M    E1
9  15.000823      59         1         3         M    E1


# there are 3 unique sender 

In [ ]:

def backtrack_all(df, hop_target=5):
    """
    Backtrack from all nodes with DENMLABEL 'M' at hop_target
    Stop when DENMLABEL == 'L'
    Ensure time of previous node < current node time
    """
    # All nodes at hop_target with DENMLABEL 'M'
    candidates = df[(df['Hopcount'] == hop_target) & (df['DENMLABEL'] == 'M')]
    if candidates.empty:
        print(f"No nodes with DENMLABEL 'M' at Hopcount {hop_target}")
        return
    
    # Loop over all nodes
    for idx, node in candidates.iterrows():
        print("\n======================================")
        print("Starting node (DENMLABEL='M'):")
        print(node[['time','Sender','receiver','DENMLABEL','Hopcount']])
        
        # If legitimate at hop_target, no backtracking needed
        if node['DENMLABEL'] == 'L':
            print("Node is legitimate. No backtracking needed.")
            continue
        
        print("Backtracking:")
        current_sender = node['Sender']
        current_hop = node['Hopcount']
        current_time = node['time']
        
        while current_hop > 0:
            prev_nodes = df[
                (df['Hopcount'] == current_hop - 1) &
                (df['receiver'] == current_sender) &
                (df['time'] < current_time)
            ]
            
            if prev_nodes.empty:
                print(f"No previous node found for sender {current_sender} at hop {current_hop-1} with time < {current_time}")
                break
            
            prev_node = prev_nodes.iloc[-1]  # take last if multiple
            print(prev_node[['time','Sender','receiver','DENMLABEL','Hopcount']])
            
            # Stop if legitimate
            if prev_node['DENMLABEL'] == 'L':
                print("Reached legitimate node. Stopping backtrack.")
                break
            
            # Update for next iteration
            current_sender = prev_node['Sender']
            current_hop = prev_node['Hopcount']
            current_time = prev_node['time']

# Example usage
backtrack_all(df)



Starting node (DENMLABEL='M'):
time         5.114129
Sender             16
receiver            8
DENMLABEL           M
Hopcount            5
Name: 1306, dtype: object
Backtracking:
time         5.100773
Sender             17
receiver           16
DENMLABEL           M
Hopcount            4
Name: 1301, dtype: object
No previous node found for sender 17 at hop 3 with time < 5.100772806487

Starting node (DENMLABEL='M'):
time         5.11413
Sender            16
receiver          17
DENMLABEL          M
Hopcount           5
Name: 1307, dtype: object
Backtracking:
time         5.100773
Sender             17
receiver           16
DENMLABEL           M
Hopcount            4
Name: 1301, dtype: object
No previous node found for sender 17 at hop 3 with time < 5.100772806487

Starting node (DENMLABEL='M'):
time         5.11413
Sender            16
receiver          17
DENMLABEL          M
Hopcount           5
Name: 1308, dtype: object
Backtracking:
time         5.100773
Sender             17
re